<a href="https://colab.research.google.com/github/powidla/ML-course/blob/main/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Deadline: 18 May 2023

### 1. AVL tree

Implement AVL tree with interface functions Insert, Delete, Search. Provide tests.

2 points

In [12]:
class AVLNode:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
        self.height = 1

class AVLTree:
    def __init__(self):
        self.root = None

    def _get_height(self, node):
        if node is None:
            return 0
        return node.height

    def _get_balance(self, node):
        if node is None:
            return 0
        return self._get_height(node.left) - self._get_height(node.right)

    def _update_height(self, node):
        node.height = 1 + max(self._get_height(node.left), self._get_height(node.right))

    def _rotate_left(self, node):
        right_child = node.right
        right_child_left_subtree = right_child.left

        right_child.left = node
        node.right = right_child_left_subtree

        self._update_height(node)
        self._update_height(right_child)

        return right_child

    def _rotate_right(self, node):
        left_child = node.left
        left_child_right_subtree = left_child.right

        left_child.right = node
        node.left = left_child_right_subtree

        self._update_height(node)
        self._update_height(left_child)

        return left_child

    def _rebalance(self, node):
        if node is None:
            return None

        self._update_height(node)

        balance = self._get_balance(node)

        if balance > 1:
            if self._get_balance(node.left) < 0:
                node.left = self._rotate_left(node.left)
            return self._rotate_right(node)

        elif balance < -1:
            if self._get_balance(node.right) > 0:
                node.right = self._rotate_right(node.right)
            return self._rotate_left(node)

        return node

    def insert(self, key):
        def _insert(node, key):
            if node is None:
                return AVLNode(key)
            elif key < node.key:
                node.left = _insert(node.left, key)
            else:
                node.right = _insert(node.right, key)

            return self._rebalance(node)

        self.root = _insert(self.root, key)

    def delete(self, key):
        def _delete(node, key):
            if node is None:
                return None
            elif key < node.key:
                node.left = _delete(node.left, key)
            elif key > node.key:
                node.right = _delete(node.right, key)
            else:
                if node.left is None:
                    return node.right
                elif node.right is None:
                    return node.left
                else:
                    min_right_subtree = self._min_value_node(node.right)
                    node.key = min_right_subtree.key
                    node.right = _delete(node.right, min_right_subtree.key)

            return self._rebalance(node)

        self.root = _delete(self.root, key)

    def _min_value_node(self, node):
        current = node

        while current.left is not None:
            current = current.left

        return current

    def search(self, key):
        current = self.root

        while current is not None:
            if current.key == key:
                return True

            elif current.key < key:
                current = current.right

            else:
                current = current.left

        return False


In [13]:
def test_avl_tree():
    tree = AVLTree()

    # Test empty tree
    assert tree.search(0) == False

    # Test insertion and search
    tree.insert(10)
    tree.insert(20)
    tree.insert(30)
    tree.insert(40)
    tree.insert(50)
    tree.insert(25)

    assert tree.search(10) == True
    assert tree.search(20) == True
    assert tree.search(30) == True
    assert tree.search(40) == True
    assert tree.search(50) == True
    assert tree.search(25) == True
    assert tree.search(15) == False

    # Test deletion and search
    tree.delete(10)
    tree.delete(20)

    assert tree.search(10) == False
    assert tree.search(20) == False
    assert tree.search(30) == True
    assert tree.search(40) == True
    assert tree.search(50) == True
    assert tree.search(25) == True
    assert tree.search(15) == False

    # Test rebalancing
    tree.insert(5)
    tree.insert(4)
    tree.insert(3)

    assert tree.search(3) == True
    assert tree.search(4) == True
    assert tree.search(5) == True

    tree.delete(50)
    tree.delete(40)
    tree.delete(30)

    assert tree.search(50) == False
    assert tree.search(40) == False
    assert tree.search(30) == False

    tree.insert(1)
    tree.insert(2)
    tree.insert(3)
    tree.insert(4)

    assert tree.search(1) == True
    assert tree.search(2) == True
    assert tree.search(3) == True
    assert tree.search(4) == True


In [14]:
test_avl_tree()

### 2. Rabin-Karp algorithm

Implement Rabin-Karp algorithm search. Implement your own function for polynomial hash calculation. Provide tests. 

2 points

In [6]:
def rabin_karp_search(pattern, text):
    pattern_hash = polynomial_hash(pattern)
    pattern_length = len(pattern)
    text_length = len(text)

    if pattern_length > text_length:
        return []

    rolling_hash = polynomial_hash(text[:pattern_length])

    matches = []

    for i in range(text_length - pattern_length + 1):
        if pattern_hash == rolling_hash:
            if pattern == text[i:i+pattern_length]:
                matches.append(i)

        if i < text_length - pattern_length:
            rolling_hash = rolling_hash - (ord(text[i]) * pow(PRIME_BASE, pattern_length - 1))
            rolling_hash = rolling_hash * PRIME_BASE + ord(text[i + pattern_length])

    return matches

def polynomial_hash(string):
    hash_value = 0

    for char in string:
        hash_value = (hash_value * PRIME_BASE + ord(char)) % PRIME_MOD

    return hash_value

PRIME_BASE = 101
PRIME_MOD = 1000000007

In [7]:
def test_rabin_karp_search():
    assert rabin_karp_search("abc", "abcdabc") == [0, 4]
    assert rabin_karp_search("abab", "ababab") == [0, 2]
    assert rabin_karp_search("aaa", "aa") == []
    assert rabin_karp_search("a", "aaaaaa") == [0, 1, 2, 3, 4, 5]
    assert rabin_karp_search("aaa", "aaaaaa") == [0, 1, 2, 3]
    assert rabin_karp_search("abc", "defg") == []

test_rabin_karp_search()